In [1]:
import requests
import lxml.html as lh
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords 
from nltk import word_tokenize
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import missingno as msno
from wordcloud import WordCloud
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
              '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']
from plotly.offline import init_notebook_mode, iplot
import nltk

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
nltk.download(['punkt','wordnet','stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
uri_pages = '?page='
tags = ['//span[@class="stareval-note"]',\
        '//div[@class="content-txt review-card-content"]',\
        '//span[@class="review-card-meta-date light"]',\
        '//*[@class="reviews-users-comment-useful js-useful-reviews"]',\
        '//*[@class="item-profil js-follow-unfollow"]']
cols = ['Note', 'Description', 'Date','nombre_good', 'nombre_follow' ]
url = 'https://www.allocine.fr/film/fichefilm-10568/critiques/spectateurs/'
page = requests.get(url)
doc = lh.fromstring(page.content)
 
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
 
    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))
    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)
 
    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
     
    return liste
 
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        print(i)
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale


In [3]:
url = 'https://www.allocine.fr/series/ficheserie-26316/critiques/'
nbPages = 21

liste_totale = getPages(nbPages, url)
liste_totale.to_csv('allocine_the_last_of_us.csv', index=False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [4]:
liste_totale['Note'] = liste_totale['Note'].str.replace(',', '.')
liste_totale['Note'] = liste_totale['Note'].apply(lambda x: float(x))

In [5]:
liste_totale.info

<bound method DataFrame.info of      Note                                        Description  \
0     4.0  J'avoue ne pas connaître le jeu vidéo, mais j'...   
1     5.0  J'ai joué et rejoué le jeu. je conseille à ceu...   
2     4.5  Bon je vais de finir le premier épisode sur Pr...   
3     5.0  Une pure merveille déjà super fan de naugthdy ...   
4     2.0  Je n'aime pas l'actrice qui joue Ellie, peu sy...   
..    ...                                                ...   
309   5.0  La série est super, Bella Ramsay joue magnifiq...   
310   5.0  J'arrive à patienter jusqu au lundi suivant po...   
311   4.0  Série top ! Ça ressemble beaucoup à twd mais e...   
312   5.0  Tout bonnement excellent : les fans sont combl...   
313   2.5  The Last of Us se dit être une "adaptation" du...   

                           Date   nombre_good        nombre_follow  key  
0    Publiée le 17 janvier 2023    34\n\n\n16  Suivre son activité    0  
1    Publiée le 17 janvier 2023  55\n\n\n\n25  Suiv

In [6]:
liste_totale.describe().T.style.set_properties(**{"background-color": "#FBA7A7", "font-size" : "17px",
                                        "color": "#ffffff", "border-radius" : "1px", "border": "1.5px solid black"})

,count,mean,std,min,25%,50%,75%,max
Note,314.000000,3.619427,1.487726,0.500000,2.500000,4.000000,5.000000,5.000000
key,314.000000,6.977707,4.316120,0.000000,3.000000,7.000000,11.000000,14.000000


In [7]:
print("Shape of the dataset: ", liste_totale.shape)
print("_______________________________________")
print("Number of null values: ")
print(liste_totale.isnull().sum())
print("_______________________________________")
print("Number of unique values: ")
print(liste_totale.nunique())

Shape of the dataset:  (314, 6)
_______________________________________
Number of null values: 
Note             0
Description      0
Date             0
nombre_good      0
nombre_follow    0
key              0
dtype: int64
_______________________________________
Number of unique values: 
Note              10
Description      313
Date              62
nombre_good      101
nombre_follow      1
key               15
dtype: int64


In [8]:
corr=liste_totale.corr()
corr.style.background_gradient(cmap='Reds')

,Note,key
Note,1.000000,0.057137
key,0.057137,1.000000


In [9]:
liste_totale['nb_mots']= liste_totale['Description'].apply(lambda x : len(str(x).split(" ")))
liste_totale['nb_carac']= liste_totale['Description'].apply(lambda x: sum(len(mot) for mot in str(x).split(" ")))
liste_totale["sentence_count"]= liste_totale['Description'].apply(lambda x: len(str(x).split(". ")))
liste_totale["avg_word_length"]= round(liste_totale['nb_carac'] / liste_totale['nb_mots']+0.5,0)
liste_totale["avg_sentence_lenght"]= round(liste_totale['nb_mots'] / liste_totale['sentence_count'],0)

liste_totale

,Note,Description,Date,nombre_good,nombre_follow,key,nb_mots,nb_carac,sentence_count,avg_word_length,avg_sentence_lenght
0,4.0,"J'avoue ne pas connaître le jeu vidéo, mais j'...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,35,183,3,6.0,12.0
1,5.0,J'ai joué et rejoué le jeu. je conseille à ceu...,Publiée le 17 janvier 2023,55\n\n\n\n25,Suivre son activité,1,488,2520,11,6.0,44.0
2,4.5,Bon je vais de finir le premier épisode sur Pr...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,174,793,13,5.0,13.0
3,5.0,Une pure merveille déjà super fan de naugthdy ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,41,188,2,5.0,20.0
4,2.0,"Je n'aime pas l'actrice qui joue Ellie, peu sy...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,57,269,2,5.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...
309,5.0,"La série est super, Bella Ramsay joue magnifiq...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,53,269,3,6.0,18.0
310,5.0,J'arrive à patienter jusqu au lundi suivant po...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,32,172,4,6.0,8.0
311,4.0,Série top ! Ça ressemble beaucoup à twd mais e...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,21,82,1,4.0,21.0
312,5.0,Tout bonnement excellent : les fans sont combl...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,33,160,2,5.0,16.0


In [10]:
def cat_summary_with_graph(dataframe, col_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentages'),
                        specs=[[{"type": "xy"}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=dataframe[col_name].value_counts().values.tolist(),
                         x=[str(i) for i in dataframe[col_name].value_counts().index],
                         text=dataframe[col_name].value_counts().values.tolist(),
                         textfont=dict(size=15),
                         name=col_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=colors,
                                     line=dict(color='#DBE6EC',
                                               width=1))),
                  row=1, col=1)
    fig.add_trace(go.Pie(labels=dataframe[col_name].value_counts().keys(),
                         values=dataframe[col_name].value_counts().values,
                         textfont=dict(size=20),
                         textposition='auto',
                         showlegend=False,
                         name=col_name,
                         marker=dict(colors=colors)),
                  row=1, col=2)

    fig.update_layout(title={'text': col_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')

    iplot(fig)
    

In [11]:
cat_cols=["Note"]
for col in cat_cols:
    cat_summary_with_graph(liste_totale,col)

In [15]:
note_count = liste_totale['Note'].value_counts().head(10)
fig = px.bar(y=note_count.values, 
             x=note_count.index, 
             color = note_count.index,
             color_discrete_sequence=px.colors.sequential.PuBuGn,
             text=note_count.values,
             title= 'Top 10 notes for the last of us',
             template= 'plotly_dark')
fig.update_layout(
    xaxis_title="Note",
    yaxis_title="Count",
    font = dict(size=20,family="Franklin Gothic"))
fig.show()

In [18]:
note_date=liste_totale.groupby('Date')[['Note']].mean().reset_index()
note_date

,Date,Note
0,Publiée le 1 avril 2023,1.750000
1,Publiée le 1 février 2023,4.150000
2,Publiée le 1 mars 2023,3.333333
3,Publiée le 10 février 2023,2.750000
4,Publiée le 11 février 2023,3.500000
...,...,...
57,Publiée le 6 mars 2023,1.500000
58,Publiée le 7 février 2023,3.875000
59,Publiée le 7 mars 2023,3.571429
60,Publiée le 8 février 2023,2.000000


In [19]:
fig=px.bar(x=note_date['Date'],
          y=note_date["Note"],
          color=note_date["Date"],
          color_discrete_sequence=px.colors.sequential.Plasma,
          text=note_date["Note"],
          title= 'Distribution AlloCiné Score by Date',
          template= 'plotly_dark')

fig.update_layout(xaxis_title="Date",
                 yaxis_title="Note",
                 font=dict(size=15, family="Franklin Gothic"))

fig.show()

In [115]:
fig=px.bar(x=note_date['Date'],
          y=note_date["Note"],
          color=note_date["Date"],
          color_discrete_sequence=px.colors.sequential.BuGn,
          text=note_date["Note"],
          title= 'Distribution AlloCiné Score by Date',
          template= 'plotly_dark')

fig.update_layout(xaxis_title="Date",
                 yaxis_title="Note",
                 font=dict(size=15, family="Franklin Gothic"))

fig.show()

In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
data = liste_totale['Description']
# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)


In [28]:
from textblob import TextBlob

# Créer une fonction pour analyser les sentiments
def analyze_sentiment(text):
    """
    Utilise TextBlob pour analyser les sentiments d'un texte donné.
    Retourne une valeur entre -1 (négatif) et 1 (positif).
    """
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Appliquer la fonction sur chaque ligne du DataFrame
liste_totale['sentiment'] = liste_totale['Description'].apply(analyze_sentiment)

# Afficher les résultats
#print(df.head())



In [50]:
df['sentiment'] = df['Description'].apply(analyze_sentiment)
df


,Note,Description,Date,nombre_good,nombre_follow,key,high_rate,sentiment
0,4.0,"J'avoue ne pas connaître le jeu vidéo, mais j'...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,0,0.62500
1,5.0,J'ai joué et rejoué le jeu. je conseille à ceu...,Publiée le 17 janvier 2023,55\n\n\n25,Suivre son activité,1,1,-0.11250
2,4.5,Bon je vais de finir le premier épisode sur Pr...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,1,0.00000
3,5.0,Une pure merveille déjà super fan de naugthdy ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,1,0.30754
4,2.0,"Je n'aime pas l'actrice qui joue Ellie, peu sy...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,0,0.00000
...,...,...,...,...,...,...,...,...
309,5.0,"La série est super, Bella Ramsay joue magnifiq...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,1,0.27381
310,5.0,J'arrive à patienter jusqu au lundi suivant po...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,1,0.00000
311,4.0,Série top ! Ça ressemble beaucoup à twd mais e...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,0,0.78125
312,5.0,Tout bonnement excellent : les fans sont combl...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,1,1.00000


In [41]:
# Charger les données
df =  pd.read_csv('allocine_the_last_of_us.csv')
# Remplacer toutes les virgules par des points dans la colonne "prix"



In [43]:
df["Note"]

0      4.0
1      5.0
2      4.5
3      5.0
4      2.0
      ... 
309    5.0
310    5.0
311    4.0
312    5.0
313    2.5
Name: Note, Length: 314, dtype: float64

In [20]:


#df['Note'] = df['Note'].astype(float)
#df['Note'] = pd.to_numeric(df['Note'], errors='coerce').fillna(0.0)

# Binariser la variable de taux (rate) en utilisant la méthode de la médiane
median_rate = liste_totale['Note'].median()
liste_totale['high_rate'] = np.where(liste_totale['Note'] > median_rate, 1, 0)


In [94]:
liste_totale

,Note,Description,Date,nombre_good,nombre_follow,key,high_rate
0,4.0,"J'avoue ne pas connaître le jeu vidéo, mais j'...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,0
1,5.0,J'ai joué et rejoué le jeu. je conseille à ceu...,Publiée le 17 janvier 2023,55\n\n\n\n25,Suivre son activité,1,1
2,4.5,Bon je vais de finir le premier épisode sur Pr...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,1
3,5.0,Une pure merveille déjà super fan de naugthdy ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,1
4,2.0,"Je n'aime pas l'actrice qui joue Ellie, peu sy...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,0
...,...,...,...,...,...,...,...
309,5.0,"La série est super, Bella Ramsay joue magnifiq...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,1
310,5.0,J'arrive à patienter jusqu au lundi suivant po...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,1
311,4.0,Série top ! Ça ressemble beaucoup à twd mais e...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,0
312,5.0,Tout bonnement excellent : les fans sont combl...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,1


In [60]:
french_stopwords=stopwords.words('french')
liste_totale['Description']=liste_totale.Description.apply(lambda x:[item for item in x if item not in french_stopwords])
liste_totale['Description']

0      [J, ', a, v, o, u, e,  , e,  , p, a,  , o, a, ...
1      [J, ', a, i,  , o, u, é,  , e,  , r, e, o, u, ...
2      [B, o,  , e,  , v, a, i,  , e,  , f, i, i, r, ...
3      [U, e,  , p, u, r, e,  , e, r, v, e, i, e,  , ...
4      [J, e,  , ', a, i, e,  , p, a,  , ', a, r, i, ...
                             ...                        
309    [L, a,  , é, r, i, e,  , e,  , u, p, e, r, ,, ...
310    [J, ', a, r, r, i, v, e,  ,  , p, a, i, e, e, ...
311    [S, é, r, i, e,  , o, p,  , !,  , Ç, a,  , r, ...
312    [T, o, u,  , b, o, e, e,  , e, x, e, e,  , :, ...
313    [T, h, e,  , L, a,  , o, f,  , U,  , e,  , i, ...
Name: Description, Length: 314, dtype: object

In [61]:
stemmer=SnowballStemmer('french')
liste_totale['Description']=liste_totale['Description'].apply(lambda x: [stemmer.stem(item) for item in x ])
liste_totale

,Note,Description,Date,nombre_good,nombre_follow,key,high_rate,sentiment
0,4.0,"[j, ', a, v, o, u, e, , e, , p, a, , o, a, ...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,0,0.62500
1,5.0,"[j, ', a, i, , o, u, é, , e, , r, e, o, u, ...",Publiée le 17 janvier 2023,55\n\n\n25,Suivre son activité,1,1,-0.11250
2,4.5,"[b, o, , e, , v, a, i, , e, , f, i, i, r, ...",Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,1,0.00000
3,5.0,"[u, e, , p, u, r, e, , e, r, v, e, i, e, , ...",Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,1,0.30754
4,2.0,"[j, e, , ', a, i, e, , p, a, , ', a, r, i, ...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,0,0.00000
...,...,...,...,...,...,...,...,...
309,5.0,"[l, a, , é, r, i, e, , e, , u, p, e, r, ,, ...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,1,0.27381
310,5.0,"[j, ', a, r, r, i, v, e, , , p, a, i, e, e, ...",Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,1,0.00000
311,4.0,"[s, é, r, i, e, , o, p, , !, , ç, a, , r, ...",Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,0,0.78125
312,5.0,"[t, o, u, , b, o, e, e, , e, x, e, e, , :, ...",Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,1,1.00000


In [62]:
liste_totale['Description']=liste_totale['Description'].apply(lambda x: ' '.join(x))


In [63]:
df

,Note,Description,Date,nombre_good,nombre_follow,key,high_rate,sentiment
0,4.0,j ' a v o u e e p a o a î r e e e u ...,Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,0,0.62500
1,5.0,j ' a i o u é e r e o u é e e u . ...,Publiée le 17 janvier 2023,55\n\n\n25,Suivre son activité,1,1,-0.11250
2,4.5,b o e v a i e f i i r e p r e i e ...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,1,0.00000
3,5.0,u e p u r e e r v e i e é u p e r f ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,1,0.30754
4,2.0,j e ' a i e p a ' a r i e q u i o u ...,Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,0,0.00000
...,...,...,...,...,...,...,...,...
309,5.0,"l a é r i e e u p e r , b e a r a a ...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,1,0.27381
310,5.0,j ' a r r i v e p a i e e r u q u a u ...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,1,0.00000
311,4.0,s é r i e o p ! ç a r e e b e b e a ...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,0,0.78125
312,5.0,t o u b o e e e x e e : e f a o ...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,1,1.00000


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(liste_totale['Description'])

In [22]:
liste_totale

,Note,Description,Date,nombre_good,nombre_follow,key,nb_mots,nb_carac,sentence_count,avg_word_length,avg_sentence_lenght,high_rate
0,4.0,"J'avoue ne pas connaître le jeu vidéo, mais j'...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0,35,183,3,6.0,12.0,0
1,5.0,J'ai joué et rejoué le jeu. je conseille à ceu...,Publiée le 17 janvier 2023,55\n\n\n\n25,Suivre son activité,1,488,2520,11,6.0,44.0,1
2,4.5,Bon je vais de finir le premier épisode sur Pr...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2,174,793,13,5.0,13.0,1
3,5.0,Une pure merveille déjà super fan de naugthdy ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3,41,188,2,5.0,20.0,1
4,2.0,"Je n'aime pas l'actrice qui joue Ellie, peu sy...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4,57,269,2,5.0,28.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
309,5.0,"La série est super, Bella Ramsay joue magnifiq...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10,53,269,3,6.0,18.0,1
310,5.0,J'arrive à patienter jusqu au lundi suivant po...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11,32,172,4,6.0,8.0,1
311,4.0,Série top ! Ça ressemble beaucoup à twd mais e...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12,21,82,1,4.0,21.0,0
312,5.0,Tout bonnement excellent : les fans sont combl...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13,33,160,2,5.0,16.0,1


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create an instance of the TfidfVectorizer class
tfidf_vectorizer = TfidfVectorizer()

# apply the vectorizer to the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(liste_totale['Description'])

# print the shape of the resulting matrix
print(tfidf_matrix)
# Récupérer les noms de fonctionnalités (mots-clés) et les scores TF-IDF associés

feature_names=vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray()


  (0, 3839)	0.25809255980203083
  (0, 4453)	0.09049176060259402
  (0, 108)	0.12786768480928498
  (0, 1642)	0.23971033015497406
  (0, 1619)	0.07563229905218063
  (0, 358)	0.304822767651347
  (0, 3105)	0.14276256189391645
  (0, 1128)	0.0930973623744194
  (0, 4309)	0.2094347708497596
  (0, 392)	0.09978748369544557
  (0, 3923)	0.18949276974323268
  (0, 4451)	0.0901739695208323
  (0, 4136)	0.15280361909298795
  (0, 2378)	0.072728801207704
  (0, 3238)	0.09547518994372124
  (0, 3327)	0.2848807665448201
  (0, 4428)	0.11696217626362157
  (0, 250)	0.26998588946018853
  (0, 3458)	0.095822803226125
  (0, 4740)	0.100161414871155
  (0, 3267)	0.15372251958879346
  (0, 667)	0.11950625329607066
  (0, 140)	0.23229217417173592
  (0, 4343)	0.11404677404817216
  (0, 2529)	0.20711024175055187
  :	:
  (313, 1283)	0.17003488564930305
  (313, 4208)	0.09658734404060251
  (313, 4465)	0.07992637987063117
  (313, 2908)	0.07938290450218181
  (313, 2400)	0.0810454093411201
  (313, 2414)	0.03969544982642868
  (313, 4

In [89]:
tfidf_matrix

<314x4797 sparse matrix of type '<class 'numpy.float64'>'
	with 23200 stored elements in Compressed Sparse Row format>

In [87]:
liste_totale

,Note,Description,Date,nombre_good,nombre_follow,key
0,4.0,"J'avoue ne pas connaître le jeu vidéo, mais j'...",Publiée le 17 janvier 2023,34\n\n\n16,Suivre son activité,0
1,5.0,J'ai joué et rejoué le jeu. je conseille à ceu...,Publiée le 17 janvier 2023,55\n\n\n\n25,Suivre son activité,1
2,4.5,Bon je vais de finir le premier épisode sur Pr...,Publiée le 17 janvier 2023,31\n\n\n22,Suivre son activité,2
3,5.0,Une pure merveille déjà super fan de naugthdy ...,Publiée le 19 janvier 2023,21\n\n\n15,Suivre son activité,3
4,2.0,"Je n'aime pas l'actrice qui joue Ellie, peu sy...",Publiée le 8 février 2023,37\n\n\n28,Suivre son activité,4
...,...,...,...,...,...,...
309,5.0,"La série est super, Bella Ramsay joue magnifiq...",Publiée le 15 mars 2023,0\n\n\n2,Suivre son activité,10
310,5.0,J'arrive à patienter jusqu au lundi suivant po...,Publiée le 23 février 2023,0\n\n\n3,Suivre son activité,11
311,4.0,Série top ! Ça ressemble beaucoup à twd mais e...,Publiée le 27 mars 2023,0\n\n\n0,Suivre son activité,12
312,5.0,Tout bonnement excellent : les fans sont combl...,Publiée le 21 février 2023,0\n\n\n2,Suivre son activité,13


In [26]:

# Sélectionner les variables d'intérêt
X = tfidf_matrix
y = liste_totale['high_rate']

# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.index)

# Créer un modèle de régression logistique
model = LogisticRegression()

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)
y_reglog = model.predict(tfidf_matrix)
# Évaluer les performances du modèle en utilisant la précision (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)




Accuracy: 0.7578947368421053


In [98]:
y_pred


array([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0])

In [108]:
import pandas as pd

# Exemple de données


# Créer le dataframe
df = pd.DataFrame({'VT': y_pred, 'VP': X_test, 'TextBlob': liste_totale["sentiment"]})

# Afficher le dataframe
print(df)


     VT  VP  TextBlob
0     1   0   0.62500
1     1   1  -0.11250
2     1   1   0.00000
3     1   1   0.30754
4     0   0   0.00000
..   ..  ..       ...
309   1   1   0.27381
310   1   1   0.00000
311   1   0   0.78125
312   1   1   1.00000
313   0   0   0.00000

[314 rows x 3 columns]
